In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import carga_datos as cd
import importlib
importlib.reload(cd)



ModuleNotFoundError: No module named 'carga_datos'

Carga de Datos

In [5]:
Data= cd.plot_excel_sheets(r"C:/Users/ekkol/OneDrive/Escritorio/RepositorioVario/lab_intermedio/difraccion_electrones/datos_difraccion_electrones.xlsx", False)
radio_pequeño = Data["Radio Pequeño"]
radio_grande = Data["Radio Grande"]
#print(Data)


NameError: name 'cd' is not defined

calculamos $\lambda_{\text{broglie}} = \frac{h}{\sqrt{2 e V m_e}}$


In [6]:
V_radio_pequeño = radio_pequeño["y"] * 1000 #Pasamos de kV a V
V_radio_grande = radio_grande["y"] * 1000 #Pasamos de kV a V
h = 6.62607015e-34 #Joules*segundos
m_e = 9.10938356e-31 #kg
e = 1.602176634e-19 #Coulombs10

def longitud_de_onda (V_v):
    l_broglie = (h / (np.sqrt(2 * m_e * e * V_v)))
    return l_broglie

def incer_lambda_broglie (V_v):
    incer_V = 0.1
    return h / (2*np.sqrt(2 * m_e * e * V_v**3)) * incer_V

#Devuelve arrays!
#print("l bragg = ",longitud_de_onda(V_radio_pequeño),"+-",incer_lambda_broglie(V_radio_pequeño))
#print("l bragg = ",longitud_de_onda(V_radio_grande),"+-",incer_lambda_broglie(V_radio_grande))

NameError: name 'radio_pequeño' is not defined

calculamos $\lambda_{\text{bragg}} = \frac{2d}{n} \sin \left[ \frac{1}{2} \tan^{-1} \left( \frac{D}{2L} \right) \right]$

In [7]:
d_1 = radio_pequeño["y"] #distancia en cm
d_2 = radio_grande["y"] #distancia en cm

#Distancias interplanares:

D1 = 213 #pm
D2 = 123 #pm

D1_m = D1 * 1e-12 #distancia en m
D2_m = D2 * 1e-12 #distancia en m

d_1_m = d_1 * 1e-2 #distancia en m
d_2_m = d_2 * 1e-2 #distancia en m

def longitud_de_onda_bragg_(d_m, D_m):
    return 2 * D_m * np.sin(0.5 * np.arctan(d_m/ (2 * 0.135)))

def incer_lambda_bragg (d_m, D_m):
    incer_d = 0.01
    return ((2 * D_m * np.cos(0.5 * np.arctan(d_m / (2 * 0.135))) * 1/(2*(1+(1/2*0.135)**2*d_m**2 * 2*0.135))) * incer_d)

print(longitud_de_onda_bragg_(0.0629, D2_m))
print(incer_lambda_bragg(d_2_m, D2_m))

NameError: name 'radio_pequeño' is not defined

Análisis de las cantidades (Ultima parte)

In [8]:
# Se conoce h

def distancia_interplanar (m):
    return (0.135*h) / (m*np.sqrt(2*m_e*e))

distancia_interplanar(3.48)

NameError: name 'h' is not defined

In [9]:
# Se conocen d_i

def hallar_h (m,d):
    return (m*d*np.sqrt(2*m_e*e)) / (2*0.135)

def incer_hallar_h (m, d):
    incer_d = 0.01
    return np.sqrt((d*np.sqrt(2*m_e*e)/(2*0.135))**2 * incer_d**2)

a = hallar_h(2.29, 123.0e-12)
b = incer_hallar_h(2.29, 123.0e-12)

print(a, b)

NameError: name 'm_e' is not defined

Definimos la función de Ajuste y Extraemos los parámetros óptimos, la matriz de covarianza. Ajustamos nuestro diccionario de valores para que sea legible por nuestra plotting function

In [10]:

def fitting_function(x, m, b):
    return m * x + b

popt_p, pcov_p = curve_fit(fitting_function, radio_pequeño["x"], radio_pequeño["y"], sigma=radio_pequeño["error"])
popt_g, pcov_g = curve_fit(fitting_function, radio_grande["x"], radio_grande["y"], sigma=radio_grande["error"])

formula_text_p = r"$y = {:.2f}x + {:.2f}$".format(popt_p[0], popt_p[1])
formula_text_g = r"$y = {:.2f}x + {:.2f}$".format(popt_g[0], popt_g[1])


radio_pequeño["popt"]=popt_p
radio_pequeño["formula_text"] = formula_text_p
radio_grande["popt"]=popt_g
radio_grande["formula_text"]=formula_text_g


cd.plot_fitted_curve(fitting_function, False, "Graficas_res" ,dataset1= radio_pequeño, dataset2 = radio_grande)

NameError: name 'radio_pequeño' is not defined